
# MVP Sprint III - Engenharia de Dados - CartolaFC
##Adriana Legal Reis##

##1. Definição do Problema

O propósito fundamental deste projeto reside na análise do desempenho dos jogadores ao longo do Campeonato Brasileiro de Futebol 2023 - Série A (Brasileirão), com o intuito de fornecer uma base sólida para as decisões de escalação nas rodadas futuras do CartolaFC.

Dentro desse contexto, temos o objetivo de responder às seguintes questões:<br>
1. Qual jogador detém a melhor média de pontos ao longo do campeonato?<br>
2. Quais são os jogadores mais consistentes em termos de pontuação, ou seja, aqueles que mantiveram um desempenho constante ao longo do campeonato?<br>
3. Qual jogador apresenta o maior número de gols marcados?<br>
4. Qual goleiro é o menos vazado?<br>
5. Qual é a relação entre o número de cartões amarelos ou vermelhos recebidos por um jogador e sua pontuação média?<br>
6. Quais são os jogadores que se destacam em assistências, ou seja, aqueles que mais contribuíram para os gols de suas equipes?<br>
7. Existe alguma correlação entre a posição do jogador (atacante, meio-campista, zagueiro, lateral e goleiro) e sua média de pontuação?<br>
8. Quais são os times que mais pontuaram no Campeonato Brasileiro e quais jogadores contribuíram significativamente para essas pontuações?<br>
9. Como o desempenho dos jogadores varia em jogos em casa versus jogos fora de casa?<br>

Este projeto visa utilizar análises estatísticas e dados históricos para oferecer insights valiosos aos gestores de equipes no CartolaFC, auxiliando-os na tomada de decisões estratégicas para o sucesso em suas escalações nas rodadas subsequentes.

In [0]:
# Importação das bibliotecas necessárias
import requests
import json
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType
from IPython.display import Image
from delta import DeltaTable
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.types import LongType
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")


## 2. Coleta dos Dados

A aquisição dos dados foi realizada por meio da API oficial do CartolaFC, seguida pelo armazenamento desses dados em formato JSON e CSV em um repositório na nuvem hospedado no ambiente do Databricks. Esse processo de coleta e armazenamento é fundamental para disponibilizar as informações necessárias para as análises subsequentes de desempenho dos jogadores no Campeonato Brasileiro de Futebol 2023 - Série A (Brasileirão), a fim de dar suporte às decisões estratégicas de escalação no CartolaFC.


**Importação das rodadas do Campeonato Brasileiro de 2023**

Utilizaremos a API específica para importar os dados das Rodadas do Campeonato.

In [0]:
# URL da API do CartolaFC
url = "https://api.cartola.globo.com/rodadas"

# Chamada à API
resposta = requests.request("GET", url)

# Verificação se a chamada foi bem-sucedida
if resposta.status_code == 200:
    objetos = json.loads(resposta.text)
    # Criando um DataFrame Pandas com os dados
    dfrod = pd.DataFrame(objetos)

    # Caminho para salvar o arquivo CSV no Databricks
    csv_path = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/rodadas.csv'
    # Salvar o DataFrame como um arquivo CSV
    dfrod.to_csv(csv_path, index=False)

    # Caminho para salvar o arquivo JSON no Databricks
    json_path_club = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/rodadas.json'
    # Salvar os DataFrames como arquivos JSON
    dfrod.to_json(json_path_club, orient='records', lines=True)
else:
    print("Erro ao acessar a API do CartolaFC")

**Carregamento os Dados das Rodadas no DataFrame dfrod**

Nesta etapa, procedemos com a carga dos dados das rodadas para o DataFrame denominado "dfrod" e sua organização em uma estrutura de dados tabular para uso nas estapas posteriores.

In [0]:
# Caminho do arquivo CSV
file_location = "/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/rodadas.csv"

# Lê o arquivo CSV usando o Pandas
dfrod = pd.read_csv(file_location)

# Exibe as primeiras linhas do DataFrame
display(dfrod)

inicio,fim,nome_rodada,rodada_id
2023-04-15 16:00:00,2023-04-16 18:30:00,Rodada 1,1
2023-04-22 16:00:00,2023-04-24 20:00:00,Rodada 2,2
2023-04-29 16:30:00,2023-05-01 20:00:00,Rodada 3,3
2023-05-06 16:00:00,2023-05-08 20:00:00,Rodada 4,4
2023-05-10 19:00:00,2023-05-11 20:00:00,Rodada 5,5
2023-05-13 16:00:00,2023-05-14 18:30:00,Rodada 6,6
2023-05-20 16:00:00,2023-05-22 20:00:00,Rodada 7,7
2023-05-27 16:00:00,2023-05-28 19:00:00,Rodada 8,8
2023-06-03 16:00:00,2023-06-05 20:00:00,Rodada 9,9
2023-06-10 16:00:00,2023-06-11 18:30:00,Rodada 10,10


__Importação das partidas de cada rodada do Campeonato__

Agora, avançaremos para a etapa de importação dos dados das partidas de cada rodada através da API específica, executando um loop para cada rodada do Campeonato Brasileiro de Futebol 2023.

In [0]:
# Cria uma sessão Spark
spark = SparkSession.builder.appName("Cartola").getOrCreate()

# Define a URL da API do CartolaFC
baseurl = "https://api.cartola.globo.com/partidas/"

# Itera sobre os valores da coluna "rodada_id" e faz as chamadas à API
for rodada_id in dfrod['rodada_id']:
    url = baseurl + str(rodada_id)

    # Faz a chamada à API
    resposta = requests.request("GET", url)

    # Verifica se a chamada foi bem-sucedida
    if resposta.status_code == 200:
        objetos = json.loads(resposta.text)
        # Crie um DataFrame Pandas com os dados
        dfpart = pd.json_normalize(objetos['partidas'])
        dfpart['rodada_id'] = rodada_id
        # Especifica o caminho para salvar o arquivo CSV no Databricks
        basecsv_path = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/partidas_rodada_'
        csv_path = basecsv_path + str(rodada_id) + ".csv"
        # Salva o DataFrame como um arquivo CSV
        dfpart.to_csv(csv_path, index=False)

        # Caminho para salvar o arquivo JSON no Databricks
        json_path_part = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/partidas_rodada_'
        
        # Salvar os DataFrames como arquivos JSON
        dfpart.to_json(json_path_part + str(rodada_id) + ".json", orient='records', lines=True)

    else:
        print(f"Erro ao acessar a API do CartolaFC Rodada " + str(rodada_id))

**Importação dos dados dos Clubes**

Utilizaremos a API específica para importar os dados dos Clubes.

In [0]:
# URL da API do CartolaFC
url = "https://api.cartola.globo.com/clubes"

# Chamada à API
resposta = requests.request("GET", url)

# Verificação se a chamada foi bem-sucedida
if resposta.status_code == 200:
    # Carrega o JSON para um dicionário
    dados_dict = json.loads(resposta.text)

    # Converte o dicionário para um DataFrame
    dfclub = pd.DataFrame(dados_dict).T

    # Especifica o caminho para salvar o arquivo CSV no Databricks
    csv_path = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/clubes.csv'
    # Salva o DataFrame como um arquivo CSV
    dfclub.to_csv(csv_path, index=False)

    # Caminho para salvar o arquivo JSON no Databricks
    json_path_club = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/clubes.json'
    # Salvar os DataFrames como arquivos JSON
    dfclub.to_json(json_path_club, orient='records', lines=True)
else:
    print("Erro ao acessar a API do CartolaFC")

**Importação dos dados da Posição dos jogadores**

A partir da API específica iremos importar os dados das posições em que podemos escalar jogadores no CartolaFC.

In [0]:
# URL da API do CartolaFC
url = "https://api.cartola.globo.com/posicoes"

# Chamada à API
resposta = requests.request("GET", url)

# Verificação se a chamada foi bem-sucedida
if resposta.status_code == 200:
    # Carrega o JSON para um dicionário
    dados_dict = json.loads(resposta.text)

    # Converte o dicionário para um DataFrame
    dfpos = pd.DataFrame(dados_dict).T

    # Especifica o caminho para salvar o arquivo CSV no Databricks
    csv_path = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/posicoes.csv'
    # Salva o DataFrame como um arquivo CSV
    dfpos.to_csv(csv_path, index=False)

    # Caminho para salvar o arquivo JSON no Databricks
    json_path_club = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/posicoes.json'
    # Salvar os DataFrames como arquivos JSON
    dfpos.to_json(json_path_club, orient='records', lines=True)
else:
    print("Erro ao acessar a API do CartolaFC")

**Importação dos dados dos Atletas**

Utilizaremos a API específica para importar os dados dos Atletas que disputam o Brasileirão 2023.

In [0]:
# URL da API do CartolaFC
url = "https://api.cartola.globo.com/atletas/mercado"

# Chamada à API
resposta = requests.request("GET", url)

# Verificação se a chamada foi bem-sucedida
if resposta.status_code == 200:
    objetos = json.loads(resposta.text)
    # Criando um DataFrame Pandas com os dados
    dfatl = pd.DataFrame(objetos['atletas'])
    
    # Caminho para salvar o arquivo CSV no Databricks
    csv_path_atl = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/atletas.csv'
    # Salvar o DataFrame como um arquivo CSV
    dfatl.to_csv(csv_path_atl, index=False)

    # Caminho para salvar o arquivo JSON no Databricks
    json_path_atl = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/atletas.json'
    # Salvar os DataFrames como arquivos JSON
    dfatl.to_json(json_path_atl, orient='records', lines=True)
else:
    print("Erro ao acessar a API do CartolaFC")

__Importação dos Dados de Pontuação dos Jogadores por Rodada de 2023__

Nessa etapa iremos importar os dados das pontuações dos jogadores através da API específica, executando novamente um loop para cada rodada do Campeonato Brasileiro de Futebol 2023. Este processo visa adquirir informações detalhadas sobre o desempenho dos jogadores em cada rodada. O loop percorrerá as diferentes rodadas do campeonato, permitindo a obtenção dos dados relevantes do CartolaFC.

In [0]:
# Cria uma sessão Spark
spark = SparkSession.builder.appName("Cartola").getOrCreate()

# Define a URL da API do CartolaFC
baseurl = "https://api.cartola.globo.com/atletas/pontuados/"

# Itera sobre os valores da coluna "rodada_id" e faz as chamadas à API
for rodada_id in dfrod['rodada_id']:
    url = baseurl + str(rodada_id)

    # Faz a chamada à API
    resposta = requests.request("GET", url)

    # Verifica se a chamada foi bem-sucedida
    if resposta.status_code == 200:
        objetos = json.loads(resposta.text)
        # Cria um DataFrame Pandas com os dados
        df = pd.DataFrame(objetos['atletas'])
        df = df.T
        df['rodada_id'] = rodada_id
        # Usando apply + pd.Series para criar colunas separadas
        dfscout = df['scout'].apply(pd.Series)

         # Juntar os DataFrames ao longo do eixo das colunas
        df = pd.concat([df, dfscout], axis=1)

        # Especifica o caminho para salvar o arquivo CSV no Databricks
        basecsv_path = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/pontuacao_rodada_'
        csv_path = basecsv_path + str(rodada_id) + ".csv"
        # Salva o DataFrame como um arquivo CSV
        df.to_csv(csv_path, index=False)

        # Caminho para salvar o arquivo JSON no Databricks
        json_path_club = '/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/pontuacao_rodada_'
        # Salvar os DataFrames como arquivos JSON
        dfclub.to_json(json_path_club + str(rodada_id) + ".json", orient='records', lines=True)
    else:
        print(f"API do CartolaFC Rodada " + str(rodada_id) + " não disponível")

API do CartolaFC Rodada 24 não disponível
API do CartolaFC Rodada 25 não disponível
API do CartolaFC Rodada 26 não disponível
API do CartolaFC Rodada 27 não disponível
API do CartolaFC Rodada 28 não disponível
API do CartolaFC Rodada 29 não disponível
API do CartolaFC Rodada 30 não disponível
API do CartolaFC Rodada 31 não disponível
API do CartolaFC Rodada 32 não disponível
API do CartolaFC Rodada 33 não disponível
API do CartolaFC Rodada 34 não disponível
API do CartolaFC Rodada 35 não disponível
API do CartolaFC Rodada 36 não disponível
API do CartolaFC Rodada 37 não disponível
API do CartolaFC Rodada 38 não disponível


__Dados salvos na Nuvem__

Na imagem abaixo, é possível visualizar os arquivos que foram inseridos no databricks através da consulta ao API do cartola.<br>

<img src="files/tables/PUC_MVP_Sprint3/Arquivos_Salvos_na_Nuvem.png" alt="Arquivos" width="800" height="600"><br>

Exemplo mostrando o conteúdo do CSV salvo:<br>

<img src="files/tables/PUC_MVP_Sprint3/Arquivos_Salvos_na_Nuvem_2.png" alt="Arquivos2" width="800" height="600"><br>

##3. Modelagem dos Dados

Neste projeto, optamos por utilizar o esquema de modelagem estrela para organizar os dados. A modelagem em estrela é uma abordagem de design de banco de dados relacional que simplifica a estrutura das tabelas e é especialmente adequada para a criação de um data warehouse.

Ao construir o modelo em estrela, uma tabela central de fatos é cercada por tabelas de dimensão que contêm informações descritivas sobre os dados na tabela de fatos. Essas tabelas de dimensão são conectadas diretamente à tabela de fatos por meio de chaves estrangeiras, facilitando a realização de consultas complexas e análises.

Utilizamos a plataforma GenMyModel para criar e visualizar a estrutura do nosso modelo em estrela. A representação visual do modelo está ilustrada abaixo:

![Modelagem](files/tables/PUC_MVP_Sprint3/Fato_Pontuacao_Rodadas.png)

Na imagem acima, é possível identificar claramente as chaves primárias, destacadas em amarelo, bem como as chaves estrangeiras, realçadas em cinza. Além disso, a representação visual também evidencia os relacionamentos essenciais entre as tabelas do nosso modelo de dados. Essa visualização facilita a compreensão das conexões fundamentais que moldam a estrutura do nosso banco de dados no esquema snowflake.


### Catálago de Dados

Abaixo apresentamos o catálogo de dados para cada tabela do nosso projeto.


__Catálogo de Dados - Rodadas__

Descrição Geral:<br>
Este conjunto de dados contém informações sobre as rodadas do Campeonato Brasileiro de Futebol - Seríe A de 2023.<br>

Metadados Principais:<br>
Nome do Conjunto de Dados: rodadas<br>
Formato: Banco de Dados SQL<br>
Data de Criação: 18/09/2023<br>
Localização: abfss://unity-catalog@stshdptbdlkc.dfs.core.windows.net/d142e1fb-3fac-44a8-a2dc-08218f92afc5/tables/96a8aa8e-db25-47b8-bbc5-b3d228a3d929<br>


![Rodadas](files/tables/PUC_MVP_Sprint3/Rodadas.png)

Linhagem dos Dados:<br>
Origem dos Dados: Os dados foram coletados diretamente do API do CartolaFC na url https://api.cartola.globo.com/rodadas.<br>

__Catálogo de Dados - Clubes__

Descrição Geral:<br>
Este conjunto de dados contém as informações dos clubes que disputam o Campeonato Brasileiro de Futebol - Seríe A de 2023.

Metadados Principais:<br>
Nome do Conjunto de Dados: clubes<br>
Formato: Banco de Dados SQL<br>
Data de Criação: 18/09/2023<br>
Localização: abfss://unity-catalog@stshdptbdlkc.dfs.core.windows.net/d142e1fb-3fac-44a8-a2dc-08218f92afc5/tables/d325f838-f9cf-42f5-ba38-960962f71764<br>


![Clubes](files/tables/PUC_MVP_Sprint3/Clube.png)

Linhagem dos Dados:<br>
Origem dos Dados: Os dados foram coletados diretamente do API do CartolaFC na url https://api.cartola.globo.com/clubes.<br>

__Catálogo de Dados - Posições__

Descrição Geral:<br>
Este conjunto de dados contém as informações das posições em que os jogadores podem ser escalados no CartolaFC.

Metadados Principais:<br>
Nome do Conjunto de Dados: posições<br>
Formato: Banco de Dados SQL<br>
Data de Criação: 18/09/2023<br>
Localização: abfss://unity-catalog@stshdptbdlkc.dfs.core.windows.net/d142e1fb-3fac-44a8-a2dc-08218f92afc5/tables/7c1e5dfe-fdf3-4811-86c5-cde753916ea5<br>


![Posições](files/tables/PUC_MVP_Sprint3/Posicao.png)

Linhagem dos Dados:<br>
Origem dos Dados: Os dados foram coletados diretamente do API do CartolaFC na url https://api.cartola.globo.com/posicoes.<br>

__Catálogo de Dados - Atletas__

Descrição Geral:<br>
Este conjunto de dados contém as informações dos jogadores dos clubes que participam o Campeonato Brasileiro de Futebol - Seríe A de 2023.

Metadados Principais:<br>
Nome do Conjunto de Dados: atletas<br>
Formato: Banco de Dados SQL<br>
Data de Criação: 18/09/2023<br>
Localização: abfss://unity-catalog@stshdptbdlkc.dfs.core.windows.net/d142e1fb-3fac-44a8-a2dc-08218f92afc5/tables/03afe76f-75ac-4ec5-8fee-e710d6155079<br>


![Atletas](files/tables/PUC_MVP_Sprint3/Atletas.png)

Linhagem dos Dados:<br>
Origem dos Dados: Os dados foram coletados diretamente do API do CartolaFC na url https://api.cartola.globo.com/atletas/mercado.<br>

__Catálogo de Dados - Partidas__

Descrição Geral:<br>
Este conjunto de dados contém as informações das partidas do Campeonato Brasileiro de Futebol - Seríe A de 2023.

Metadados Principais:<br>
Nome do Conjunto de Dados: partidas<br>
Formato: Banco de Dados SQL<br>
Data de Criação: 18/09/2023<br>
Localização: abfss://unity-catalog@stshdptbdlkc.dfs.core.windows.net/d142e1fb-3fac-44a8-a2dc-08218f92afc5/tables/ad5cc4f3-a1f1-415a-bfb1-28b98c16c36e<br>


![Partidas](files/tables/PUC_MVP_Sprint3/Partidas.png)

Linhagem dos Dados:<br>
Origem dos Dados: Os dados foram coletados diretamente do API do CartolaFC na url https://api.cartola.globo.com/partidas/.<br>

__Catálogo de Dados - Pontuação__

Descrição Geral:<br>
Este conjunto de dados contém as informações das pontuações que os jogadores obtiveram em cada partida do Campeonato Brasileiro de Futebol - Seríe A de 2023.

Metadados Principais:<br>
Nome do Conjunto de Dados: pontuacao<br>
Formato: Banco de Dados SQL<br>
Data de Criação: 18/09/2023<br>
Localização: abfss://unity-catalog@stshdptbdlkc.dfs.core.windows.net/d142e1fb-3fac-44a8-a2dc-08218f92afc5/tables/c662a04a-47c8-4e5c-acb9-dbc00c1d806a<br>


![Pontuação](files/tables/PUC_MVP_Sprint3/Pontuacao-1.png)

Linhagem dos Dados:<br>
Origem dos Dados: Os dados foram coletados diretamente do API do CartolaFC na url https://api.cartola.globo.com/atletas/pontuados/.<br>

###Criação do Banco de Dados


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dt0028_dev.PUC_MVP_CartolaFC;

Podemos verificar a criação do banco de dados vazio na imagem abaixo .<br>


<img src="files/tables/PUC_MVP_Sprint3/Banco_de_Dados.png" alt="Banco_de_Dados" width="800" height="600"><br>

<img src="files/tables/PUC_MVP_Sprint3/Banco_de_Dados_2.png" alt="Banco_Dados2" width="400" height="300"><br>


###Criação das Tabelas

In [0]:
%sql

USE dt0028_dev.PUC_MVP_CartolaFC;

-- Criação da tabela 'rodadas'
CREATE OR REPLACE TABLE rodadas
(
  rodada_id BIGINT NOT NULL PRIMARY KEY,
  inicio TIMESTAMP,
  fim TIMESTAMP,
  nome_rodada VARCHAR(10)
);

-- Criação da tabela 'clubes'
CREATE OR REPLACE TABLE clubes
(
  clube_id BIGINT NOT NULL PRIMARY KEY,
  nome VARCHAR(255),
  abreviacao VARCHAR(60),
  slug VARCHAR(60),
  apelido VARCHAR(60),
  nome_fantasia VARCHAR(60)
);

-- Criação da tabela 'atletas'
CREATE OR REPLACE TABLE atletas
(
  atleta_id BIGINT NOT NULL,
  jogos_num INTEGER,
  slug VARCHAR(60),
  apelido VARCHAR(60) PRIMARY KEY,
  apelido_abreviado VARCHAR(60),
  nome VARCHAR(255)
);

-- Criação da tabela 'posicao'
CREATE OR REPLACE TABLE posicao
(
  posicao_id BIGINT NOT NULL PRIMARY KEY,
  abreviacao VARCHAR(3),
  nome VARCHAR(10)
);

-- Criação da tabela 'partidas'
CREATE OR REPLACE TABLE partidas
(
  partida_id BIGINT NOT NULL PRIMARY KEY,
  local VARCHAR(150),
  partida_data TIMESTAMP,
  placar_oficial_visitante INTEGER,
  placar_oficial_mandante INTEGER,
  clube_visitante_posicao INTEGER,
  clube_casa_posicao INTEGER,
  clube_visitante_id BIGINT,
  clube_casa_id BIGINT,
  valida BOOLEAN,
  rodada_id BIGINT,
  CONSTRAINT partidas_clubes_visitante_fk FOREIGN KEY (clube_visitante_id) REFERENCES clubes(clube_id),
  CONSTRAINT partidas_clubes_casa_fk FOREIGN KEY (clube_casa_id) REFERENCES clubes(clube_id),
  FOREIGN KEY (rodada_id) REFERENCES rodadas(rodada_id)
);

-- Criação da tabela 'pontuacao'
CREATE OR REPLACE TABLE pontuacao
(
  apelido VARCHAR(60),
  pontuacao DECIMAL(4, 2),
  posicao_id BIGINT,
  clube_id BIGINT,
  entrou_em_campo BOOLEAN,
  partida_id BIGINT,
  rodada_id BIGINT,
  CA INTEGER,
  DS INTEGER,
  FC INTEGER,
  FF INTEGER,
  FD INTEGER,
  FS INTEGER,
  I INTEGER,
  SG INTEGER,
  A INTEGER,
  G INTEGER,
  DE INTEGER,
  GS INTEGER,
  V INTEGER,
  PS INTEGER,
  FT INTEGER,
  PP INTEGER,
  DP INTEGER,
  CV INTEGER,
  PC INTEGER,
  PRIMARY KEY (apelido, rodada_id),
  FOREIGN KEY (apelido) REFERENCES atletas(apelido),
  FOREIGN KEY (posicao_id) REFERENCES posicao(posicao_id),
  FOREIGN KEY (clube_id) REFERENCES clubes(clube_id),
  FOREIGN KEY (rodada_id) REFERENCES rodadas(rodada_id),
  FOREIGN KEY (partida_id) REFERENCES partidas(partida_id)
);

In [0]:
%sql
SHOW TABLES IN dt0028_dev.puc_mvp_cartolafc;

database,tableName,isTemporary
puc_mvp_cartolafc,atletas,false
puc_mvp_cartolafc,clubes,false
puc_mvp_cartolafc,partidas,false
puc_mvp_cartolafc,pontuacao,false
puc_mvp_cartolafc,posicao,false
puc_mvp_cartolafc,rodadas,false


Podemos verificar na imagem abaixo que as tabelas foram devidamente criadas no diretório do Databrincks.<br>

<img src="files/tables/PUC_MVP_Sprint3/Tabelas_Criadas.png" alt="Tabelas" width="800" height="600"><br>


Abaixo estão as imagens de cada tabela criada com suas chaves primárias e estrangeiras:<br>

**Tabela Rodadas**<br>
<img src="files/tables/PUC_MVP_Sprint3/Tabela_Rodadas.png" alt="Rodada" width="400" height="300"><br>

**Tabela Clubes**<br>
<img src="files/tables/PUC_MVP_Sprint3/Tabela_Clubes.png" alt="Clubes" width="400" height="300"><br>

**Tabela Atletas**<br>
<img src="files/tables/PUC_MVP_Sprint3/Tabela_Atletas-2.png" alt="Atletas" width="400" height="300"><br>

**Tabela Posição**<br>
<img src="files/tables/PUC_MVP_Sprint3/Tabela_Posicao.png" alt="Posição" width="400" height="300"><br>

**Tabela Partidas**<br>
<img src="files/tables/PUC_MVP_Sprint3/Tabela_Partidas.png" alt="Partidas" width="400" height="300"><br>

**Tabela Pontuação**<br>
<img src="files/tables/PUC_MVP_Sprint3/Tabela_Pontuacao.png" alt="Pontuação" width="400" height="300"><br>

## 4. Carga dos Dados

Para executar o processo de carregamento de dados em nossas tabelas de banco de dados, é fundamental adotar um procedimento bem definido. Esse procedimento engloba várias etapas, incluindo a leitura de arquivos nos formatos CSV ou JSON, a conversão desses dados em tabelas Delta e, posteriormente, a inserção seletiva dos campos que identificamos como pertinentes para nossa operação. Essa abordagem garante que apenas os dados relevantes sejam incorporados ao nosso ambiente de armazenamento, otimizando assim a eficiência e a qualidade do nosso sistema.


Importando os dados para a **Tabela Rodadas**

In [0]:
# Inicialize a sessão Spark
spark = SparkSession.builder.getOrCreate()

# Caminho do arquivo CSV
file_location = "/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/rodadas.csv"

# Lê o arquivo CSV usando o Pandas
dfrodadas = pd.read_csv(file_location)
dfrodadas['inicio'] = pd.to_datetime(dfrodadas['inicio'])
dfrodadas['fim'] = pd.to_datetime(dfrodadas['fim'])

# Converta o DataFrame Pandas em um DataFrame Spark
dfspark = spark.createDataFrame(dfrodadas)

# Escreva o DataFrame Spark diretamente na tabela Delta
dfspark.write.mode("overwrite").saveAsTable("dt0028_dev.puc_mvp_cartolafc.rodadas")

Importando os dados para a **Tabela Atletas**

In [0]:
from pyspark.sql import SparkSession
import pandas as pd

# Inicialize a sessão Spark
spark = SparkSession.builder.getOrCreate()

# Caminho do arquivo CSV
file_location = "/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/atletas.csv"

# Leia o arquivo CSV usando o Pandas
df_pandas = pd.read_csv(file_location)

# Crie um DataFrame Spark a partir do DataFrame Pandas
df_spark = spark.createDataFrame(df_pandas)

# Selecione as colunas desejadas
colunas_selecionadas = ["atleta_id", "jogos_num", "slug", "apelido", "apelido_abreviado", "nome"]
df_spark = df_spark.select(*colunas_selecionadas)
df_spark = df_spark.withColumn("jogos_num", col("jogos_num").cast(IntegerType()))

# Escreva o novo DataFrame Spark diretamente na tabela Delta
df_spark.write.mode("overwrite").saveAsTable("dt0028_dev.puc_mvp_cartolafc.atletas")

Importando os dados para a **Tabela Clubes**

In [0]:
from pyspark.sql import SparkSession
import pandas as pd

# Inicialize a sessão Spark
spark = SparkSession.builder.getOrCreate()

# Caminho do arquivo CSV
file_location = "/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/clubes.csv"

# Lê o arquivo CSV usando o Pandas
dfclubes = pd.read_csv(file_location)

# Converta o DataFrame Pandas em um DataFrame Spark
df_spark = spark.createDataFrame(dfclubes)
# Renomeie a coluna 'id' para 'posicao_id'
df_spark = df_spark.withColumnRenamed("id", "clube_id")

# Selecione as colunas desejadas
colunas_selecionadas = ["clube_id", "nome", "abreviacao", "slug", "apelido", "nome_fantasia"]
df_spark = df_spark.select(*colunas_selecionadas)

# Escreva o DataFrame Spark diretamente na tabela Delta com mesclagem automática de esquema
df_spark.write.mode("overwrite").saveAsTable("dt0028_dev.puc_mvp_cartolafc.clubes")

Importando os dados para a **Tabela Posição**


In [0]:
# Inicialize a sessão Spark
spark = SparkSession.builder.getOrCreate()

# Caminho do arquivo CSV
file_location = "/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/posicoes.csv"

# Lê o arquivo CSV usando o Pandas
dfposicoes = pd.read_csv(file_location)

# Converta o DataFrame Pandas em um DataFrame Spark
df_spark = spark.createDataFrame(dfposicoes)
# Renomeie a coluna 'id' para 'posicao_id'
df_spark = df_spark.withColumnRenamed("id", "posicao_id")

# Selecione as colunas desejadas
colunas_selecionadas = ["posicao_id", "abreviacao", "nome"]
df_spark = df_spark.select(*colunas_selecionadas)

# Escreva o DataFrame Spark diretamente na tabela Delta com mesclagem automática de esquema
df_spark.write.mode("overwrite").saveAsTable("dt0028_dev.puc_mvp_cartolafc.posicao")

Importando os dados para a **Tabela Partidas**

In [0]:
# Inicialize a sessão Spark
spark = SparkSession.builder.getOrCreate()

# Caminho do arquivo CSV
diretorio = "/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/"

# Lista de arquivos no diretório que começam com "partidas_" e têm extensão ".csv"
arquivos_no_diretorio = [arquivo for arquivo in os.listdir(diretorio) if arquivo.startswith("partidas_") and arquivo.endswith(".csv")]

# Lista para armazenar os DataFrames de cada arquivo
dataframes = []

# Para cada arquivo na lista filtrada
for arquivo in arquivos_no_diretorio:
    # Crie o caminho completo para o arquivo
    caminho_arquivo = os.path.join(diretorio, arquivo)
    
    # Leia o arquivo CSV usando o Pandas
    df = pd.read_csv(caminho_arquivo)

    # Adicione o DataFrame à lista
    dataframes.append(df)

# Concatene todos os DataFrames em um único DataFrame
dfpartidas = pd.concat(dataframes, ignore_index=True)
dfpartidas['partida_data'] = pd.to_datetime(dfpartidas['partida_data'])

# Converta o DataFrame Pandas em um DataFrame Spark
dfspark = spark.createDataFrame(dfpartidas)

# Selecione as colunas desejadas
dfspark = dfspark.select(
    "partida_id",
    "local",
    "partida_data",
    col("placar_oficial_visitante").cast("int").alias("placar_oficial_visitante"),
    col("placar_oficial_mandante").cast("int").alias("placar_oficial_mandante"),
    col("clube_visitante_posicao").cast("int").alias("clube_visitante_posicao"),
    col("clube_casa_posicao").cast("int").alias("clube_casa_posicao"),
    "clube_visitante_id",
    "clube_casa_id",
    "valida",
    "rodada_id"
)

# Escreva o DataFrame Spark diretamente na tabela Delta com mesclagem automática de esquema
dfspark.write.option("mergeSchema", "true").mode("overwrite").saveAsTable("dt0028_dev.puc_mvp_cartolafc.partidas")

Importando os dados para a **Tabela Pontuação**

In [0]:
# Inicialize a sessão Spark
spark = SparkSession.builder.getOrCreate()

# Caminho do arquivo CSV
diretorio = "/Workspace/Users/adriana.legal@petrobras.com.br/Arquivos/"

# Lista de arquivos no diretório que começam com "pontuacoes_" e têm extensão ".csv"
arquivos_no_diretorio = [arquivo for arquivo in os.listdir(diretorio) if arquivo.startswith("pontuacao_rodada_") and arquivo.endswith(".csv")]

# Lista para armazenar os DataFrames de cada arquivo
dataframes = []

# Para cada arquivo na lista filtrada
for arquivo in arquivos_no_diretorio:
    # Crie o caminho completo para o arquivo
    caminho_arquivo = os.path.join(diretorio, arquivo)
    
    # Leia o arquivo CSV usando o Pandas
    df = pd.read_csv(caminho_arquivo)

    # Adicione o DataFrame à lista
    dataframes.append(df)

# Concatene todos os DataFrames em um único DataFrame
dfpontuacoes = pd.concat(dataframes, ignore_index=True)

# Converta o DataFrame Pandas em um DataFrame Spark
dfspark = spark.createDataFrame(dfpontuacoes)
# Use a função cast para converter a coluna 'pontuacao' para decimal(4,2)
dfspark = dfspark.withColumn("pontuacao", col("pontuacao").cast("decimal(4,2)"))

# Selecione as colunas desejadas
dfspark = dfspark.select(
    "apelido",
    "pontuacao",
    "posicao_id",
    "clube_id",
    "entrou_em_campo",
    "rodada_id", 
    col("CA").cast("int").alias("CA"),
    col("DS").cast("int").alias("DS"),
    col("FC").cast("int").alias("FC"),
    col("FF").cast("int").alias("FF"),
    col("FD").cast("int").alias("FD"),
    col("FS").cast("int").alias("FS"),
    col("I").cast("int").alias("I"),
    col("SG").cast("int").alias("SG"),
    col("A").cast("int").alias("A"),
    col("G").cast("int").alias("G"),
    col("DE").cast("int").alias("DE"),
    col("GS").cast("int").alias("GS"),
    col("V").cast("int").alias("V"),
    col("PS").cast("int").alias("PS"),
    col("FT").cast("int").alias("FT"),
    col("PP").cast("int").alias("PP"),
    col("DP").cast("int").alias("DP"),
    col("CV").cast("int").alias("CV"),
    col("PC").cast("int").alias("PC"),
)

# Escreva o DataFrame Spark diretamente na tabela Delta com mesclagem automática de esquema
dfspark.write.option("mergeSchema", "true").mode("overwrite").saveAsTable("dt0028_dev.puc_mvp_cartolafc.pontuacao")

A tabela "pontuacao", que foi importada através da API, não inclui o campo "partida_id". Portanto, foi necessário preencher esse campo por meio da consulta à tabela "partidas" para obter as informações correspondentes.

In [0]:
%sql

USE dt0028_dev.PUC_MVP_CartolaFC;

MERGE INTO pontuacao AS p
USING partidas AS pt
ON p.rodada_id = pt.rodada_id
   AND (p.clube_id = pt.clube_visitante_id OR p.clube_id = pt.clube_casa_id)
WHEN MATCHED THEN
    UPDATE SET p.partida_id = pt.partida_id;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
7541,7541,0,0


## 5. Análise dos Dados

### a. Qualidade de dados

É essencial realizar uma análise de qualidade para cada atributo presente em nosso conjunto de dados. Esta análise visa identificar qualquer problema ou inconsistência nos dados que possa impactar as respostas às perguntas que buscamos resolver.

Ao realizar essa análise, podemos detectar problemas como dados ausentes, valores inconsistentes, ou erros de formatação. Para resolver esses problemas, podemos adotar estratégias como o preenchimento de valores ausentes com informações relevantes, a correção de erros de digitação, ou a padronização de formatos.

A qualidade dos dados desempenha um papel fundamental em nossas análises, pois dados confiáveis e bem-estruturados são essenciais para obter insights precisos e confiáveis.<br>

Para iniciar nossa análise, começaremos avaliando as dimensões de nossas tabelas.

In [0]:
%sql

USE dt0028_dev.PUC_MVP_CartolaFC;

-- Visualize as dimensões dos datasets
SELECT
  'rodadas' AS tabela,
  COUNT(*) AS total_linhas,
  COUNT(DISTINCT rodada_id) AS qtde
FROM
  dt0028_dev.PUC_MVP_CartolaFC.rodadas

UNION ALL

SELECT
  'partidas' AS tabela,
  COUNT(*) AS total_linhas,
  COUNT(DISTINCT partida_id) AS qtde
FROM
  dt0028_dev.PUC_MVP_CartolaFC.partidas

UNION ALL

SELECT
  'posicao' AS tabela,
  COUNT(*) AS total_linhas,
  COUNT(DISTINCT posicao_id) AS qtde
FROM
  dt0028_dev.PUC_MVP_CartolaFC.posicao

UNION ALL

SELECT
  'clubes' AS tabela,
  COUNT(*) AS total_linhas,
  COUNT(DISTINCT clube_id) AS qtde
FROM
  dt0028_dev.PUC_MVP_CartolaFC.clubes

UNION ALL

SELECT
  'atletas' AS tabela,
  COUNT(*) AS total_linhas,
  COUNT(DISTINCT atleta_id) AS qtde
FROM
  dt0028_dev.PUC_MVP_CartolaFC.atletas

UNION ALL

SELECT
  'pontuacao' AS tabela,
  COUNT(*) AS total_linhas,
  COUNT(DISTINCT apelido) AS qtde
FROM
  dt0028_dev.PUC_MVP_CartolaFC.pontuacao


tabela,total_linhas,qtde
rodadas,38,38
partidas,380,380
posicao,6,6
clubes,67,67
atletas,780,780
pontuacao,7541,673


No resultado apresnentado na tabela acima já podemos verificar algumas informações importantes.

A quantidade apresentada de rodadas está correta, pois o Campeonato Brasileiro possui 38 rodadas. A quantidade de Partidas também está coerente, pois temos 10 partidas em cada rodada, que dá as 380 quantificadas na tabela.

A tabela de posição possui 6 dados, que são as posições que utilizamos para fazer a escalçao no cartolaFC. Podemos conferir exibindo as informações da tabela posição:

In [0]:
%sql
-- Exibir todas as informações da tabela posicao
SELECT *
FROM dt0028_dev.PUC_MVP_CartolaFC.posicao;


posicao_id,abreviacao,nome
5,ata,Atacante
6,tec,Técnico
1,gol,Goleiro
2,lat,Lateral
3,zag,Zagueiro
4,mei,Meia


A quantidade distinta de times apresentada na tabela clubes não está correspondendo ao esperado. Pois a quantidade de clubes que disputam o campeonato é de 20 times. Podemos verificar isso se fizemos uma junção da tabela pontuação e clubes e verificarmos os clube que estão com pontuação zero.

In [0]:
%sql
-- Junte a tabela pontuacao com a tabela clubes e filtre clubes com pontuação igual a zero ou null
SELECT
  c.nome AS Clube,  
  SUM(COALESCE(p.pontuacao, 0)) AS Pontuacao_Clube
FROM
  dt0028_dev.PUC_MVP_CartolaFC.clubes c
LEFT JOIN
  dt0028_dev.PUC_MVP_CartolaFC.pontuacao p
ON
  p.clube_id = c.clube_id
GROUP BY
  c.nome
HAVING
  SUM(COALESCE(p.pontuacao, 0)) = 0
ORDER BY
  Clube ASC

Clube,Pontuacao_Clube
ABC,0.00
ASA Arapiraca,0.00
Americana,0.00
América-RN,0.00
Atlético-GO,0.00
Avaí,0.00
Boa Esporte,0.00
Botafogo-PB,0.00
Botafogo-SP,0.00
Brasil de Pelotas,0.00


Podemos verificar que nossa consulta resultou em 47 clubes com pontuação zero, confirmando, portanto, que temos apenas os 20 clubes do campeonato com pontuação válida na tabela pontuação.<br>

Podemos efetuar a mesma análise para validar a tabela de atletas.

In [0]:
%sql
-- Calcule a pontuação média por atleta na tabela 'pontuacao' e classifique por maior pontuação média
WITH PontuacaoPorAtleta AS (
  SELECT
    p.apelido AS Jogador,
    a.nome As Nome,
    c.nome AS Clube,  
    SUM(COALESCE(p.pontuacao, 0)) AS Pontuacao
  FROM
    dt0028_dev.PUC_MVP_CartolaFC.pontuacao p
  JOIN
    dt0028_dev.PUC_MVP_CartolaFC.atletas a
  ON
    p.apelido = a.apelido
  JOIN
    dt0028_dev.PUC_MVP_CartolaFC.clubes c
  ON
    p.clube_id = c.clube_id
  GROUP BY
    c.nome, p.apelido, a.nome
)

SELECT
  Jogador,
  Nome,
  Clube,
  Pontuacao
FROM
  PontuacaoPorAtleta
WHERE
  Clube IN (
    SELECT DISTINCT
      c.nome
    FROM
      dt0028_dev.PUC_MVP_CartolaFC.clubes c
    LEFT JOIN
      dt0028_dev.PUC_MVP_CartolaFC.pontuacao p
    ON
      c.clube_id = p.clube_id
    GROUP BY
      c.nome
    HAVING
      SUM(COALESCE(p.pontuacao, 0)) = 0
  )
ORDER BY
  Clube ASC


Jogador,Nome,Clube,Pontuacao


O resultado "A consulta não retornou resultados" indica que a tabela de Atletas contém apenas jogadores dos times que estão atualmente participando do Campeonato Brasileiro de 2023.

Vamos agora exibir as informações de cada tabela do Banco de Dados e um exemplo dos dados contindos nelas.

**Tabela Rodadas**

In [0]:
%sql
-- Exibir os tipos de dados da tabela 'rodadas'
DESCRIBE dt0028_dev.PUC_MVP_CartolaFC.rodadas;

col_name,data_type,comment
rodada_id,bigint,null
inicio,timestamp,null
fim,timestamp,null
nome_rodada,varchar(10),null


In [0]:
%sql
-- Exibir os primeiros dados da tabela 'rodadas'
SELECT * FROM dt0028_dev.PUC_MVP_CartolaFC.rodadas LIMIT 5;

rodada_id,inicio,fim,nome_rodada
1,2023-04-15T16:00:00.000+0000,2023-04-16T18:30:00.000+0000,Rodada 1
2,2023-04-22T16:00:00.000+0000,2023-04-24T20:00:00.000+0000,Rodada 2
3,2023-04-29T16:30:00.000+0000,2023-05-01T20:00:00.000+0000,Rodada 3
4,2023-05-06T16:00:00.000+0000,2023-05-08T20:00:00.000+0000,Rodada 4
5,2023-05-10T19:00:00.000+0000,2023-05-11T20:00:00.000+0000,Rodada 5


**Tabela Partidas**

In [0]:
%sql
-- Exibir os tipos de dados da tabela 'partidas'
DESCRIBE dt0028_dev.PUC_MVP_CartolaFC.partidas;

col_name,data_type,comment
partida_id,bigint,null
local,varchar(150),null
partida_data,timestamp,null
placar_oficial_visitante,int,null
placar_oficial_mandante,int,null
clube_visitante_posicao,int,null
clube_casa_posicao,int,null
clube_visitante_id,bigint,null
clube_casa_id,bigint,null
valida,boolean,null


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Exibir os primeiros dados da tabela 'partidas'
SELECT *
FROM dt0028_dev.PUC_MVP_CartolaFC.partidas
LIMIT 5;

partida_id,local,partida_data,placar_oficial_visitante,placar_oficial_mandante,clube_visitante_posicao,clube_casa_posicao,clube_visitante_id,clube_casa_id,valida,rodada_id
302573,Allianz Parque,2023-04-15T16:00:00.000+0000,1,2,10,2,1371,275,true,1
302580,Independência,2023-04-15T16:00:00.000+0000,3,0,6,19,266,327,true,1
302572,Nilton Santos (Engenhão),2023-04-15T18:30:00.000+0000,1,2,13,1,276,263,true,1
302575,Nabi Abi Chedid,2023-04-15T18:30:00.000+0000,1,2,15,3,265,280,true,1
302578,Arena da Baixada,2023-04-15T18:30:00.000+0000,0,2,16,7,290,293,true,1


**Tabela Posição**

In [0]:
%sql
-- Exibir os tipos de dados da tabela 'posicao'
DESCRIBE dt0028_dev.PUC_MVP_CartolaFC.posicao;

col_name,data_type,comment
posicao_id,bigint,null
abreviacao,varchar(3),null
nome,varchar(10),null


In [0]:
%sql
-- Exibir os primeiros dados da tabela 'posicao'
SELECT *
FROM dt0028_dev.PUC_MVP_CartolaFC.posicao
LIMIT 6;

posicao_id,abreviacao,nome
5,ata,Atacante
6,tec,Técnico
1,gol,Goleiro
2,lat,Lateral
3,zag,Zagueiro
4,mei,Meia


**Tabela Clubes**

In [0]:
%sql
-- Exibir os tipos de dados da tabela 'clubes'
DESCRIBE dt0028_dev.PUC_MVP_CartolaFC.clubes;

col_name,data_type,comment
clube_id,bigint,null
nome,varchar(255),null
abreviacao,varchar(60),null
slug,varchar(60),null
apelido,varchar(60),null
nome_fantasia,varchar(60),null


In [0]:
%sql
-- Exibir os primeiros dados da tabela 'clubes'
SELECT *
FROM dt0028_dev.PUC_MVP_CartolaFC.clubes
LIMIT 5;

clube_id,nome,abreviacao,slug,apelido,nome_fantasia
1,Outros,OUT,NaN,NaN,Outros
1349,Ipatinga,IPA,NaN,NaN,Ipatinga
1371,Cuiabá,CUI,cuiaba,Dourado,Cuiabá
1390,Icasa,ICA,NaN,NaN,Icasa
2190,Oeste,OES,NaN,NaN,Oeste


**Tabela Atletas**

In [0]:
%sql
-- Exibir os tipos de dados da tabela 'atletas'
DESCRIBE dt0028_dev.PUC_MVP_CartolaFC.atletas;

col_name,data_type,comment
atleta_id,bigint,null
jogos_num,int,null
slug,varchar(60),null
apelido,varchar(60),null
apelido_abreviado,varchar(60),null
nome,varchar(255),null


In [0]:
%sql
-- Exibir os primeiros dados da tabela 'atletas'
SELECT *
FROM dt0028_dev.PUC_MVP_CartolaFC.atletas
LIMIT 5;

atleta_id,jogos_num,slug,apelido,apelido_abreviado,nome
99552,0,rossi,Rossi,Rossi,Agustín Daniel Rossi
83817,7,clayson,Clayson,Clayson,Clayson Henrique da Silva Vieira
125470,10,thiago-kosloski,Thiago Kosloski,T. Kosloski,Thiago Kosloski
125471,5,serginho,Serginho,Serginho,Sérgio Antonio de Luiz Junior
82627,7,luiz-araujo,Luiz Araújo,L. Araújo,Luiz de Araújo Guimarães Neto


**Tabela Pontuação**

In [0]:
%sql
-- Exibir os tipos de dados da tabela 'pontuacao'
DESCRIBE dt0028_dev.PUC_MVP_CartolaFC.pontuacao;

col_name,data_type,comment
apelido,varchar(60),null
pontuacao,"decimal(4,2)",null
posicao_id,bigint,null
clube_id,bigint,null
entrou_em_campo,boolean,null
partida_id,bigint,null
rodada_id,bigint,null
CA,int,null
DS,int,null
FC,int,null


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Exibir os primeiros dados da tabela 'pontuacao'
SELECT *
FROM dt0028_dev.PUC_MVP_CartolaFC.pontuacao
LIMIT 5;

apelido,pontuacao,posicao_id,clube_id,entrou_em_campo,partida_id,rodada_id,CA,DS,FC,FF,FD,FS,I,SG,A,G,DE,GS,V,PS,FT,PP,DP,CV,PC
Khellven,4.20,2,293,true,302715,7,0,3,3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
Bento,0.00,1,293,true,302715,7,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0
Gabriel Xavier,2.90,3,265,true,302719,7,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Matheus Bidu,3.50,2,264,true,302711,7,0,1,1,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
Rikelme,2.50,5,1371,true,302716,7,0,2,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0


A partir dessas análises, podemos validar nossas tabelas, com a exceção da tabela "clubes" que contém informações sobre times que não estão atualmente participando do campeonato. Essa informação não interfere nas análises a serem realizadas, uma vez que as tabelas de "jogadores" e "pontuações" não incluem dados desses times. No entanto, por motivos didáticos, optamos por realizar uma limpeza da tabela "clubes".

In [0]:
%sql
-- Crie uma tabela temporária para identificar os clubes com pontuação igual a zero
CREATE OR REPLACE TEMPORARY VIEW clubes_a_excluir AS
SELECT
  c.clube_id
FROM
  dt0028_dev.PUC_MVP_CartolaFC.clubes c
LEFT JOIN
  dt0028_dev.PUC_MVP_CartolaFC.pontuacao p
ON
  p.clube_id = c.clube_id
GROUP BY
  c.clube_id
HAVING
  SUM(COALESCE(p.pontuacao, 0)) = 0;

-- Exclua os clubes com base na tabela temporária
DELETE FROM dt0028_dev.PUC_MVP_CartolaFC.clubes
WHERE clube_id IN (SELECT clube_id FROM clubes_a_excluir);

num_affected_rows
47


Podemos exibir agora os dados da tabela clubes que terá apenas o s 20 clubes que disputamo campeonato de 2023.

In [0]:
%sql
-- Exibir os primeiros dados da tabela 'clubes'
SELECT *
FROM dt0028_dev.PUC_MVP_CartolaFC.clubes

clube_id,nome,abreviacao,slug,apelido,nome_fantasia
1371,Cuiabá,CUI,cuiaba,Dourado,Cuiabá
262,Flamengo,FLA,flamengo,Mengão,Flamengo
263,Botafogo,BOT,botafogo,Glorioso,Botafogo
264,Corinthians,COR,corinthians,Timão,Corinthians
265,Bahia,BAH,bahia,Tricolor de Aço,Bahia
266,Fluminense,FLU,fluminense,Tricolor,Fluminense
267,Vasco,VAS,vasco,Gigante da Colina,Vasco
275,Palmeiras,PAL,palmeiras,Verdão,Palmeiras
276,São Paulo,SAO,sao-paulo,Tricolor,São Paulo
277,Santos,SAN,santos,Peixe,Santos


Databricks visualization. Run in Databricks to view.

### b. Solução do problema

A partir desse momento vamos fazer as consultas e gerar os gráficos e as análises necessárias para responder as questões formuladas no nosso iniciado.<br>
<br>
1. Qual jogador detém a melhor média de pontos ao longo do campeonato?<br>


In [0]:
%sql
-- Calcule a pontuação média por atleta na tabela 'pontuacao' e classifique por maior pontuação média
SELECT
  p.apelido AS Jogador,
  po.nome AS Posicao,
  c.nome AS Clube,  
  ROUND(AVG(p.pontuacao), 2) AS Pontuacao_Media,
  SUM(CASE WHEN p.entrou_em_campo = true THEN 1 ELSE 0 END) AS Qtde_Jogos
FROM
  dt0028_dev.PUC_MVP_CartolaFC.pontuacao p
JOIN
  dt0028_dev.PUC_MVP_CartolaFC.posicao po
ON
  p.posicao_id = po.posicao_id
JOIN
  dt0028_dev.PUC_MVP_CartolaFC.clubes c
ON
  p.clube_id = c.clube_id
GROUP BY
  p.apelido, po.nome, c.nome
ORDER BY
  pontuacao_media DESC

Jogador,Posicao,Clube,Pontuacao_Media,Qtde_Jogos
Slimani,Atacante,Coritiba,10.00,1
Lúcio Flávio,Técnico,Botafogo,9.73,1
Zé Ricardo,Técnico,Cruzeiro,9.25,1
Vegetti,Atacante,Vasco,9.07,6
Maicon,Zagueiro,Santos,8.60,1
Tiquinho Soares,Atacante,Botafogo,8.46,19
Luis Suárez,Atacante,Grêmio,8.43,18
Marlon,Lateral,Cruzeiro,8.40,22
Hulk,Atacante,Atlético-MG,8.39,20
Rossi,Atacante,Vasco,8.25,2


Databricks visualization. Run in Databricks to view.

2. Qual goleiro é o menos vazado?<br>
3. Qual jogador apresenta o maior número de gols marcados?<br>
4. Quais são os jogadores mais consistentes em termos de pontuação, ou seja, aqueles que mantiveram um desempenho constante ao longo do campeonato?<br>
5. Qual é a relação entre o número de cartões amarelos ou vermelhos recebidos por um jogador e sua pontuação média?<br>
6. Quais são os jogadores que se destacam em assistências, ou seja, aqueles que mais contribuíram para os gols de suas equipes?<br>
7. Existe alguma correlação entre a posição do jogador (atacante, meio-campista, zagueiro, lateral e goleiro) e sua média de pontuação?<br>
8. Quais são os times que mais pontuaram no Campeonato Brasileiro e quais jogadores contribuíram significativamente para essas pontuações?<br>
9. Como o desempenho dos jogadores varia em jogos em casa versus jogos fora de casa?<br>

In [0]:
%sql
-- Calcule a pontuação média por atleta na tabela 'pontuacao' e classifique por maior pontuação média
SELECT
  p.apelido AS Jogador,
  po.nome AS Posicao,
  c.nome AS Clube,  
  ROUND(AVG(p.pontuacao), 2) AS Pontuacao_Media,
  SUM(G) AS Gols,
  SUM(A) AS Assistencias,
  SUM(FT + FF + FD) AS Finalizacoes,
  SUM(DS) AS Desarmes,
  SUM(DE) AS Defesas,
  Sum(SG) AS SG
FROM
  dt0028_dev.PUC_MVP_CartolaFC.pontuacao p
JOIN
  dt0028_dev.PUC_MVP_CartolaFC.posicao po
ON
  p.posicao_id = po.posicao_id
JOIN
  dt0028_dev.PUC_MVP_CartolaFC.clubes c
ON
  p.clube_id = c.clube_id
GROUP BY
  p.apelido, po.nome, c.nome
ORDER BY
  pontuacao_media DESC

Jogador,Posicao,Clube,Pontuacao_Media,Gols,Assistencias,Finalizacoes,Desarmes,Defesas,SG
Slimani,Atacante,Coritiba,10.00,0,1,4,1,0,0
Lúcio Flávio,Técnico,Botafogo,9.73,0,0,0,0,0,0
Zé Ricardo,Técnico,Cruzeiro,9.25,0,0,0,0,0,0
Vegetti,Atacante,Vasco,9.07,4,1,9,4,0,0
Maicon,Zagueiro,Santos,8.60,0,0,1,2,0,1
Tiquinho Soares,Atacante,Botafogo,8.46,13,4,28,8,0,0
Luis Suárez,Atacante,Grêmio,8.43,6,5,49,10,0,0
Marlon,Lateral,Cruzeiro,8.40,2,2,12,88,0,9
Hulk,Atacante,Atlético-MG,8.39,7,5,60,8,0,0
Rossi,Atacante,Vasco,8.25,0,2,0,6,0,0


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Calcule a pontuação média por atleta na tabela 'pontuacao' e classifique por maior pontuação média
SELECT
  p.apelido AS Jogador,
  po.nome AS Posicao,
  c.nome AS Clube,  
  p.rodada_id AS Rodada,
 SUM(pontuacao) AS Pontuacao,
  SUM(G) AS Gols,
  SUM(A) AS Assistencias,
  SUM(FT + FF + FD) AS Finalizacoes,
  SUM(DS) AS Desarmes,
  SUM(DE) AS Defesas,
  Sum(SG) AS SG
FROM
  dt0028_dev.PUC_MVP_CartolaFC.pontuacao p
JOIN
  dt0028_dev.PUC_MVP_CartolaFC.posicao po
ON
  p.posicao_id = po.posicao_id
JOIN
  dt0028_dev.PUC_MVP_CartolaFC.clubes c
ON
  p.clube_id = c.clube_id
GROUP BY
  p.apelido, po.nome, c.nome, p.rodada_id
ORDER BY
  pontuacao DESC

Jogador,Posicao,Clube,Rodada,Pontuacao,Gols,Assistencias,Finalizacoes,Desarmes,Defesas,SG
Raphael Veiga,Meia,Palmeiras,5,27.60,2,1,3,2,0,0
Gustavo Sauer,Atacante,Botafogo,17,24.60,2,1,0,3,0,0
Lucero,Atacante,Fortaleza,21,24.40,2,0,3,1,0,0
Marcos Leonardo,Atacante,Santos,14,22.50,2,1,2,0,0,0
Moisés,Atacante,Fortaleza,3,22.20,2,0,5,0,0,0
Guilherme Marques,Atacante,Goiás,14,21.90,2,1,3,1,0,0
Paulinho,Atacante,Atlético-MG,21,21.30,2,0,2,3,0,0
Pavón,Atacante,Atlético-MG,5,21.10,1,2,2,1,0,0
Mendoza,Atacante,Santos,3,20.80,1,1,1,4,0,0
Léo Pereira,Zagueiro,Flamengo,3,20.80,2,0,0,3,0,0
